In [1]:
# Импорт библиотек
import os
import time
import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from calculate_CCGT_PKM import Calculate_CCGT_PKM_iter
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
accumulation = pd.read_excel("blocks.xlsx", sheet_name="accumulation", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWIN", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWIN", "T"]
water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################

# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 20
Maxiterations_cotel = 10
Maxiterations_turbine = 20

# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True


# Расчет для работы с теплофикацией
# Teplo = int(False)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = list(
    map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures)
)


#####################Максимов#####################
water_streams.at["SWIN", "H"] = water.p_t(1, water_streams.at["SWIN", "T"])["h"]
water_streams.at["SWOUT", "H"] = water.p_t(1, water_streams.at["SWOUT", "T"])["h"]

# Время зарядки/разрядки, часы
time_ac = 4
# Время ожидания, часы
time_jdat = 12
# Конструкция аккумулятора
constr = {
    "Diametr": 10,
    "kolichestvo": 1,
    "Visota": 15,
    "lambda_min_vata": 0.045,
    "delta_min_vata": 0.01,
}

syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")

Iter_pkm = 20
pkm_pgu_tol = 10**-2




In [2]:
from calculate_CCGT_PKM import Calculate_CCGT_PKM_iter

PKM_zaryad = False
PKM_razryad = True

arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
    constr,
    time_jdat,
]
New_Iter_pkm=20
pkm_pgu_tol=10**-2
CCGT=Calculate_CCGT_PKM_iter(arguments_all_it, New_Iter_pkm, pkm_pgu_tol)

Раход на СГтурбину турбину =0, всё ушло на осн ГТУ
Qgvto: 0.0
Тепла в ГВТО не хватает на теплофикацию. Расход на СП: 750.0
Время 1 итерации расчета КУ+ТУ:---  27.1 сек. ---
Перепад давлений на ЦНД равен:  -2.1327573327192817e-09
Перепад давлений на ЦНД равен:  -0.0005440756278933733
Достигнуто максимальное количество итераций давления верхнего отбора
Diafragma_it [0.009932420811158485, 0.010452823155170488, 0.010854257493507604, 0.01061182782129466]
Погрешность давления в верхнем сетевом отборе в %:  0.727
Достигнуто максимальное количество итераций давления верхнего отбора
Diafragma_it [0.012200077745231332, 0.012392815443350372, 0.01150572995542041, 0.010720468535945098]
Погрешность давления в верхнем сетевом отборе в %:  2.356
Достигнуто максимальное количество итераций давления верхнего отбора
Diafragma_it [0.012423694390289397, 0.011667752169193689, 0.011574185627812678, 0.011510502533018442]
Погрешность давления в верхнем сетевом отборе в %:  0.191
Время 2 итерации расчета КУ+ТУ:

In [3]:
water_streams

,T,P,H,G,S,X
AIR,8.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,508.336367,8.381084,3415.659112,63.204456,NaN,NaN
IVD-PEVD,301.079087,8.719226,2747.536309,63.204456,NaN,NaN
EVD-IVD,298.997554,8.719226,1339.106569,63.204456,NaN,NaN
PEN-EVD,163.464090,8.719226,695.183964,63.204456,NaN,NaN
BND-PEN,162.000400,0.650333,684.170615,63.204456,NaN,NaN
PPND-DROSND,175.445636,0.644112,2792.540603,15.346326,NaN,NaN
IND-PPND,162.000400,0.650333,2759.621188,15.346326,NaN,NaN
GPK-IND,158.179193,0.650333,667.603612,78.550782,NaN,NaN
GPK-REC,158.179193,0.650333,667.603612,93.607453,NaN,NaN


In [4]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
COOL-HTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HTS-HTSCOOL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HTSCOOL-Separ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Separ-SGaccum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SGaccum-Separacc,8.493303,2.0,1082.910511,8572.874884,0.0,0.0,0.168738,0.0,0.050357,0.097508,0.681387,0.002009
Separacc-Sepout,8.493303,2.0,965.683707,7886.573769,0.0,0.0,0.177673,0.0,0.000074,0.102671,0.717467,0.002115
Sepout-COMB,8.493303,2.0,965.683707,0.000000,0.0,0.0,0.177673,0.0,0.000074,0.102671,0.717467,0.002115
Sepout-GTU,8.493303,2.0,965.683707,0.547679,0.0,0.0,0.177673,0.0,0.000074,0.102671,0.717467,0.002115
AIR-COMP,0.000000,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
heaters

,Qw,Qg,KPD
PEVD,42227.692674,42654.235024,0.99
IVD,89010.844790,89909.944232,0.99
EVD,40698.812762,41109.911881,0.99
PPND,505.150231,510.252759,0.99
IND,33150.962335,33485.820540,0.99
GPK,38930.768874,39324.008964,0.99
SP2,86.081119,NaN,0.99
SP1,74.120334,NaN,0.99
OD,6.535717,NaN,0.99
Strans,0.000000,NaN,NaN


In [6]:
accumulation

,Qw,T,V,Qnr
ASW,NaN,NaN,NaN,NaN
PKM,9.283656e+06,8.493303,NaN,NaN


In [7]:
from PKM import PKM_all

Gst = 1
pkm = PKM_all.calc(True,gas_streams,syngas_streams,water_streams,water_streams0,heaters, Gst)

TypeError: PKM_all.calc() takes 6 positional arguments but 7 were given

In [ ]:
accumulation